In [11]:
import pyshark
import glob, os
import numpy as np
import pandas as pd
import itertools
import time
from collections import Counter

In [2]:
# Global Variables
ROUTER = '78:d2:94:4d:ab:3e'
WIFI_DEVICES = ['ec:1a:59:e4:fd:41', 'ec:1a:59:e4:fa:09',
                'ec:1a:59:e5:02:0d', '14:91:82:24:dd:35',
                '60:38:e0:ee:7c:e5', '14:91:82:cd:df:3d',
                'b4:75:0e:0d:94:65', 'b4:75:0e:0d:33:d5',
                '94:10:3e:2b:7a:55', '30:8c:fb:3a:1a:ad',
                'd0:73:d5:26:b8:4c', 'd0:73:d5:26:c9:27',
                'ac:84:c6:97:7c:cc', 'b0:4e:26:c5:2a:41',
                '70:4f:57:f9:e1:b8', ROUTER]

DEVICE_TYPE = {'ec:1a:59:e4:fd:41' : 'camera',
               'ec:1a:59:e4:fa:09' : 'camera',
               'ec:1a:59:e5:02:0d' : 'camera',
               '14:91:82:24:dd:35' : 'plug',
               '60:38:e0:ee:7c:e5' : 'plug',
               '14:91:82:cd:df:3d' : 'plug',
               'b4:75:0e:0d:94:65' : 'plug',
               'b4:75:0e:0d:33:d5' : 'plug',
               '94:10:3e:2b:7a:55' : 'plug',
               '30:8c:fb:3a:1a:ad' : 'camera',
               'd0:73:d5:26:b8:4c' : 'bulb', 
               'd0:73:d5:26:c9:27' : 'bulb',
               'ac:84:c6:97:7c:cc' : 'camera', 
               'b0:4e:26:c5:2a:41' : 'bulb',
               '70:4f:57:f9:e1:b8' : 'plug',
                ROUTER : 'router'}

DEVICE_NAME = {'ec:1a:59:e4:fd:41' : 'Netcam1', 
               'ec:1a:59:e4:fa:09' : 'Netcam2',
               'ec:1a:59:e5:02:0d' : 'Netcam3',
               '14:91:82:24:dd:35' : 'Insight',
               '60:38:e0:ee:7c:e5' : 'Mini',
               '14:91:82:cd:df:3d' : 'Switch1',
               'b4:75:0e:0d:94:65' : 'Switch2',
               'b4:75:0e:0d:33:d5' : 'Switch3',
               '94:10:3e:2b:7a:55' : 'Switch4',
               '30:8c:fb:3a:1a:ad' : 'Dropcam',
               'd0:73:d5:26:b8:4c' : 'Lifx1', 
               'd0:73:d5:26:c9:27' : 'Lifx2',
               'ac:84:c6:97:7c:cc' : 'Kasa', 
               'b0:4e:26:c5:2a:41' : 'TpBulb',
               '70:4f:57:f9:e1:b8' : 'TpPlug',
                ROUTER : 'Router'}

TRAINING_TEST = {'ec:1a:59:e4:fd:41' : 'train', 
                 'ec:1a:59:e4:fa:09' : 'train',
                 'ec:1a:59:e5:02:0d' : 'test',
                 '14:91:82:24:dd:35' : 'train',
                 '60:38:e0:ee:7c:e5' : 'train',
                 '14:91:82:cd:df:3d' : 'train',
                 'b4:75:0e:0d:94:65' : 'train',
                 'b4:75:0e:0d:33:d5' : 'train',
                 '94:10:3e:2b:7a:55' : 'test',
                 '30:8c:fb:3a:1a:ad' : 'train',
                 'd0:73:d5:26:b8:4c' : 'train', 
                 'd0:73:d5:26:c9:27' : 'test',
                 'ac:84:c6:97:7c:cc' : 'test', 
                 'b0:4e:26:c5:2a:41' : 'train',
                 '70:4f:57:f9:e1:b8' : 'test',}

DATA_PKT_SUBTYPES = {32 : 'Data',
                     40 : 'QoS_Data',
                     44 : 'QoS_Null'}

FEATURES = ["Time", "PacketLength", "Duration", "SourceAddr", "DestAddr", "Subtype"]
SRC_DIR = './Source/'
DST_DIR = './Destination/'
PCAP_DIR = '/root/Documents/Thesis/PCAPS'
TIMING_PKT_NUMBER = 25000
DATA_FRAME_TYPE = '2'

path_name = os.getcwd()
DATE = path_name[path_name.rindex('/')+1:]
PROC_TIME = "wifi_processing_time_" + DATE + ".csv"

In [3]:
def count_assoc_pkts(device):
    # Get the count of packets of a given device that are sent within a second of each other (associated packets)
    
    ASSOC_PKT_THRESHOLD = 1 # the threshold in seconds within which a packet will be considered an assoc_pkt

    # Extract time values of all packets belonging to a certain device
    df_device = df[df["Name"]==device]
    pkt_time_values = np.array(df_device["Time"].values)
    print "Length =", len(pkt_time_values)
    
    assoc_pkt_counts = []
    
    # Iterate through each packet of the device
    for pkt_index in range(0,len(df_device)):  

        # Create an array of size=len(pkt_time_values) that contains the time value of packet X
        pkt_time = np.full((len(pkt_time_values),),df_device.iloc[pkt_index]["Time"])

        # Calculate the time difference between packet X and all other packets
        diff = np.abs(np.subtract(pkt_time, pkt_time_values))

        # Calculate the count of packets that would be considered an assoc_pkt based on ASSOC_PKT_THRESHOLD
        assoc_pkts = (diff <= ASSOC_PKT_THRESHOLD).sum()
        assoc_pkt_counts.append(assoc_pkts)
        
    
    series = pd.Series(assoc_pkt_counts, index=df_device.index)
    return series

#         print "Count assoc_pkts = ", assoc_pkts

In [6]:
# cap = pyshark.FileCapture('/root/Documents/Thesis/PCAPS/master.cap', only_summaries=False)
cap = pyshark.FileCapture('/root/Documents/Thesis/BLE_PCAPS/push-5min.pcap')

SCAN_RSP = '4'
s_adv_addr = 'd8:10:ed:43:60:ac'

In [9]:
def ble_cap(cap, field_name, field_val=None):
    time_start = time.time()
    cnt = Counter()
    search_pkt = set()
    
    for pkt in cap:
        if hasattr(pkt.btle, field_name):
            field = getattr(pkt.btle, field_name)
            cnt[field] += 1  
            if field == field_val:
                search_pkt.add(pkt)
    
    print time.time()-time_start
    return cnt, search_pkt

In [13]:
cnt, search_pkt = ble_cap(pyshark.FileCapture('/root/Documents/Thesis/BLE_PCAPS/push-5min.pcap'),
                          'advertising_address', s_adv_addr)

77.4832549095


In [12]:
accessaddr_cnt = Counter()
advaddr_cnt = Counter()

for pkt in search_pkt:
    if hasattr(pkt.btle, "access_address"):
        accessaddr_cnt[pkt.btle.access_address] += 1
    if hasattr(pkt.btle, "advertising_address"):
        advaddr_cnt[pkt.btle.advertising_address] += 1

print accessaddr_cnt.most_common()
print advaddr_cnt.most_common()


[('0x8e89bed6', 934)]
[('d8:10:ed:43:60:ac', 934)]


In [145]:
most_c = cnt.most_common(10)
device_names = [device[0] for device in most_c]

device_names.remove('L402EL4')
device_names.remove('danalock-B129586')

device_names

['Eve Energy 556E',
 'Eve Energy 51C0',
 'Kevo',
 'Eve Weather 943D',
 'Eve Room 8F24',
 'Eve Room 4A04',
 'Eve Door 91B3',
 'Eve Door DC42']